In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import cycler
import BeamDynamics as bd
import copy

In [ ]:
from importlib import reload
reload(bd)

In [ ]:
# %matplotlib inline
# %matplotlib notebook
%matplotlib widget
plt.rcParams['figure.figsize'] = [9.6, 6.4]
defaultColorCycle = plt.rcParams["axes.prop_cycle"].by_key()['color']
# plotFont = {
#     'family' : 'sans-serif',
#     'weight' : 'normal',
#     'size'   : 12
# }
# matplotlib.rc('font', **plotFont)
# plt.rc('legend', fontsize=10)

# Gamma FLASH Therapy

## Plot Drive Electron Beam at E = 35 MeV

In [ ]:
sdfFilePath = '/afs/psi.ch/project/newgun/gammaFlashTherapy/E35MeV_D5mm/FCCeeTargetTracking_primary.root.sdf_txt'
driveBeam = bd.load_standard_fwf(sdfFilePath)
totElectronsIn = driveBeam.shape[0]
driveBeam.describe()

## Optimize Gamma Fluence with Target Thickness at E = 35 MeV

In [ ]:
# sdfFilePath = '/afs/psi.ch/project/newgun/gammaFlashTherapy/E35MeV_D20mm/FCCeeTargetTracking_amor_leave.root.sdf_txt'
# sdfFilePath = '/afs/psi.ch/project/newgun/gammaFlashTherapy/E35MeV_D10mm/FCCeeTargetTracking_amor_leave.root.sdf_txt'
sdfFilePath = '/afs/psi.ch/project/newgun/gammaFlashTherapy/E35MeV_D3mm/FCCeeTargetTracking_amor_leave.root.sdf_txt'
targetOut = bd.load_standard_fwf(sdfFilePath)
targetOut.describe()

In [ ]:
driveBeamE = 35.  # [MeV]
EperChargeDriveBeam = driveBeamE / 1e3  # [J/nC]
targetThicknessList = np.arange(1, 16, 1)
simCollection = []
pdgIdList = []
for targetThickness in targetThicknessList:
    sdfFilePath = '/afs/psi.ch/project/newgun/gammaFlashTherapy/E{:.0f}MeV_D{:.0f}mm/FCCeeTargetTracking_amor_leave.root.sdf_txt'.format(
        driveBeamE, targetThickness
    )
    distr = bd.load_standard_fwf(sdfFilePath)
    particleCounts = {}
    EparticleType = {}
    for pdgId in distr['pdgId'].unique():
        if pdgId not in pdgIdList:
            pdgIdList.append(pdgId)
        selIds = distr['pdgId'].isin([pdgId])
        particleCounts[pdgId] = distr[selIds].shape[0]
        # TODO: Check Ekin vs. E!!!
        EparticleType[pdgId] = distr[selIds]['Ekin'].sum() / totElectronsIn / 1e3  # [J/nC]
    simCollection.append({
        'driveBeamE': driveBeamE,
        'targetThickness': targetThickness,
        'distr': distr,
        'partCounts': particleCounts,
        'EparticleType': EparticleType
    })

In [ ]:
pdgIdList

In [ ]:
partNames = {
    11: 'Electrons',
    22: 'Photons',
    -11: 'Positrons'
}

In [ ]:
selectedSims = [sim for sim in simCollection if sim['driveBeamE'] == driveBeamE]
partCounts = {}
EsingleTypes = {}
selTargetThicknesses = [sim['targetThickness'] for sim in selectedSims]
for pdgId in pdgIdList:
    partCountSingleType = []
    EsingleType = []
    for sim in selectedSims:
        try:
            partCountSingleType.append(sim['partCounts'][pdgId])
            EsingleType.append(sim['EparticleType'][pdgId])
        except KeyError:
            partCountSingleType.append(0.)
            EsingleType.append(0.)
    partCounts[pdgId] = partCountSingleType
    EsingleTypes[pdgId] = EsingleType

In [ ]:
fig, ax = plt.subplots()
axr = ax.twinx()
for pdgId in pdgIdList:
    ax.plot(selTargetThicknesses, partCounts[pdgId], 'o-')
    axr.plot(selTargetThicknesses, EsingleTypes[pdgId], 'o--')
ax.set_xlabel('Target thickness [mm]')
ax.set_ylabel('Counts (solid lines)')
axr.set_ylabel('Average E per charge [J/nC] (dashed lines)')
ax.legend([partNames[pdgId] for pdgId in pdgIdList])
ax.set_title('E per charge drive beam = {:.3f} J/nC'.format(EperChargeDriveBeam))
ax.grid()

## Photon Spectrum with Edrive = 35 MeV

In [ ]:
EbinWidth = .2  # [MeV]
Ebins = np.arange(0., driveBeamE, EbinWidth)
fig, ax = plt.subplots(len(pdgIdList), 1)
customCycler = (
    cycler.cycler(color=[plt.get_cmap('jet')(1. * ind/len(selectedSims)) for ind in range(len(selectedSims))])  # +
    # cycler(lw=[1, 2, 3, 4])
)
for ind, pdgId in enumerate(pdgIdList):
    ax[ind].set_prop_cycle(customCycler)
    for sim in selectedSims:
        ax[ind].hist(sim['distr'][sim['distr']['pdgId'].isin([pdgId])]['Ekin'], bins=Ebins, label='D = {:.1f} mm'.format(sim['targetThickness']))
    ax[ind].set_xlabel('E [MeV]')
    ax[ind].set_ylabel('Counts')
    if ind == len(pdgIdList) - 1:
        ax[ind].legend()
    ax[ind].set_title(partNames[pdgId])
    ax[ind].grid()

<div class="alert alert-block alert-success">
Some good news.
</div>

<div class="alert alert-block alert-warning">
Some warning.
</div>

<div class="alert alert-block alert-danger">
Some danger.
</div>